In [3]:
import json
import numpy as np
import pandas as pd

import ngsidekick as ngsk

# Neuroglancer State

Neuroglancer links encode a JSON object that captures nearly all viewer state: layer visibility, camera orientation, etc.

Let's start with the default [MaleCNS scene][1].

[1]: https://neuroglancer-demo.appspot.com/#!gs://flyem-male-cns/v0.9/male-cns-v0.9.json

In [4]:
state = ngsk.parse_nglink('https://neuroglancer-demo.appspot.com/#!gs://flyem-male-cns/v0.9/male-cns-v0.9.json')

Most of the top-level keys control the global viewer settings: Coordinate space, camera orientation, zoom level, panel layout, etc.

In [6]:
list(state.keys())

['title',
 'dimensions',
 'position',
 'crossSectionScale',
 'projectionOrientation',
 'projectionScale',
 'layers',
 'showSlices',
 'gpuMemoryLimit',
 'systemMemoryLimit',
 'selectedLayer',
 'layout',
 'selection',
 'layerListPanel']

Neuroglancer stores the layers as a list (to preserve their order), but for programmatic manipulation it's convenient to access them by their `name`.  Here's a convenience function for that:

In [ ]:
seg_layer = ngsk.layer_state(state, 'cns-seg')

In [ ]:
# Select two neurons.
seg_layer['segments'] = ["10009", "10157"]

A (non-shortened) neuroglancer link includes the complete JSON state directly in the URL.  (It might be pretty long!)

To turn our state into a URL, just URL-encode the JSON text and give it an appropriate prefix.

In [9]:
import urllib

encoded_state = urllib.parse.quote(json.dumps(state))

print("https://neuroglancer-demo.appspot.com/#!" + encoded_state)

https://neuroglancer-demo.appspot.com/#!%7B%22title%22%3A%20%22MaleCNS%20v0.9%22%2C%20%22dimensions%22%3A%20%7B%22x%22%3A%20%5B8e-09%2C%20%22m%22%5D%2C%20%22y%22%3A%20%5B8e-09%2C%20%22m%22%5D%2C%20%22z%22%3A%20%5B8e-09%2C%20%22m%22%5D%7D%2C%20%22position%22%3A%20%5B48311.5%2C%2025295.765625%2C%2028511.5%5D%2C%20%22crossSectionScale%22%3A%2088.13122564876332%2C%20%22projectionOrientation%22%3A%20%5B0%2C%20-0.002181659685447812%2C%200%2C%200.999997615814209%5D%2C%20%22projectionScale%22%3A%2099656.51830908704%2C%20%22layers%22%3A%20%5B%7B%22type%22%3A%20%22image%22%2C%20%22source%22%3A%20%7B%22url%22%3A%20%22precomputed%3A//gs%3A//flyem-male-cns/em/em-clahe-jpeg%22%2C%20%22subsources%22%3A%20%7B%22default%22%3A%20true%7D%2C%20%22enableDefaultSubsources%22%3A%20false%7D%2C%20%22tab%22%3A%20%22rendering%22%2C%20%22name%22%3A%20%22em-clahe%22%7D%2C%20%7B%22type%22%3A%20%22segmentation%22%2C%20%22source%22%3A%20%5B%7B%22url%22%3A%20%22precomputed%3A//gs%3A//flyem-male-cns/v0.9/segmentation%2

It can be cumbersome to share long links, and not all standard link shorteners can handle neuroglancer links with complete fidelity. Fortunately, neuroglancer can also unpack the JSON state from a JSON file (as shown above in the link we used to load the initial state).  However, that requires hosting the JSON file somewhere, either over `https` or in a Google Storage bucket (prefixed with `gs://`).

Janelia FlyEM offers a [neuroglancer link shortener][shortng] that stores your link in a GCS bucket (with caveats).  In addition to the Web UI, it has a REST API as shown here.

[shortng]: https://shortng-bmcp5imp6q-uc.a.run.app/shortener.html

(If you prefer to use your own GCS bucket, see the convenience functions in [`ngsidekick.gcs`][gcs].)

[gcs]: https://janelia-flyem.github.io/ngsidekick/docs/gcs.html

In [ ]:
import requests

SHORTENER_URL = 'https://shortng-bmcp5imp6q-uc.a.run.app'

r = requests.post(f'{SHORTENER_URL}/shortng', json={
        'state': state,

        # Optional: A default name is chosen if you don't provide one.
        'filename': 'my-state.json',

        # Optional: A password to prevent others from writing to the same filename.
        # Note: Does NOT prevent others from using your link!
        'password': 'my-password',
    })

print(r.json())

{'link': 'https://neuroglancer-demo.appspot.com/#!gs://flyem-user-links/short/my-state.json'}


# Download Example data

MaleCNS neuron properties and soma locations.

Obtained from the [MaleCNS download page.][1]

[1]: https://male-cns.janelia.org/download

In [37]:
body_annotations_url = 'https://storage.googleapis.com/flyem-male-cns/v0.9/connectome-data/flat-connectome/body-annotations-male-cns-v0.9-minconf-0.5.feather'
df = pd.read_feather(body_annotations_url)

# The "type" column has the cell type, but later on we'll use "type" to mean something else,
# so rename "type" -> "cell_type"
df = df.rename(columns={'type': 'cell_type'})

print(df.columns)
df.head()

Index(['bodyId', 'flywireType', 'group', 'hemibrainType', 'instance',
       'itoleeHl', 'somaSide', 'statusLabel', 'superclass', 'supertype',
       'cell_type', 'class', 'entryNerve', 'fruDsx', 'matchingNotes',
       'rootSide', 'assignedOlHex1', 'assignedOlHex2', 'mancBodyid',
       'mancGroup', 'mancType', 'somaNeuromere', 'subclass', 'trumanHl',
       'dimorphism', 'synonyms', 'birthtime', 'mancSerial', 'mcnsSerial',
       'exitNerve', 'serialMotif', 'receptorType', 'somaLocation',
       'tosomaLocation', 'status'],
      dtype='object')


,bodyId,flywireType,group,hemibrainType,instance,itoleeHl,somaSide,statusLabel,superclass,supertype,...,synonyms,birthtime,mancSerial,mcnsSerial,exitNerve,serialMotif,receptorType,somaLocation,tosomaLocation,status
0,10001,DNp01,10001.0,Giant Fiber,DNp01(GF)_R,putative_primary,R,Roughly traced,descending_neuron,None,...,Kennedy and Broadie 2018: GF,early,NaN,NaN,None,None,None,"[37124, 22258, 36274]",None,Traced
1,10002,OCG01d,10002.0,OCG01,OCG01d_L,None,L,Roughly traced,visual_projection,None,...,None,None,NaN,NaN,None,None,None,"[50214, 10279, 27497]",None,Traced
2,10003,VCH,10003.0,VCH,VCH_R,putative_primary,R,Prelim Roughly traced,visual_centrifugal,None,...,None,early,NaN,NaN,None,None,None,"[46716, 21029, 15714]",None,Traced
3,10005,AOTU019,10005.0,AOTU019,AOTU019_R,putative_primary,R,Roughly traced,cb_intrinsic,None,...,None,early,NaN,NaN,None,None,None,"[41190, 18491, 12885]",None,Traced
4,10006,"VS1,VS2,VS3,VS4,VS5,VS6,VS7,VS8",10006.0,VS,VS_L,no_lineage,L,Prelim Roughly traced,visual_projection,10006,...,None,None,NaN,NaN,None,None,None,"[69604, 28368, 42459]",None,Traced


In [28]:
# Extract the rows with a soma location
soma_df = df.dropna(subset='somaLocation').copy()
soma_df[[*'xyz']] = soma_df['somaLocation'].tolist()

print(soma_df.shape)
soma_df[['bodyId', *'xyz']].head()


(141780, 38)


,bodyId,x,y,z
0,10001,37124,22258,36274
1,10002,50214,10279,27497
2,10003,46716,21029,15714
3,10005,41190,18491,12885
4,10006,69604,28368,42459


# Local Annotations
To visualize just a few points, neuroglancer allows "local" annotations -- stored directly in the neuroglancer link (JSON state).

In [30]:
# Neuroglancer allows local annotation layers to contain a mix of annotation types (points, lines, boxes, etc.)
Tm9_soma_df = soma_df.query('cell_type == "Tm9"')
Tm9_soma_layer_json = ngsk.local_annotation_json(Tm9_soma_df)

# The output below can be pasted into the neuroglancer state.
print(json.dumps(Tm9_soma_layer_json))

{"name": "annotations", "type": "annotation", "source": {"url": "local://annotations", "transform": {"outputDimensions": {"x": [8e-09, "m"], "y": [8e-09, "m"], "z": [8e-09, "m"]}}}, "tool": "annotatePoint", "shader": "void main() {\n    //\n    // Point Marker API\n    //\n    setPointMarkerSize(8.0);\n    setPointMarkerColor(vec4(defaultColor(), 1.0));\n    setPointMarkerBorderWidth(1.0);\n    setPointMarkerBorderColor(defaultColor());\n}\n", "annotations": [{"type": "point", "id": "0x2a7934f02e0d6823", "point": [75339, 18690, 24056]}, {"type": "point", "id": "0x4b9370cd71e8816a", "point": [82763, 46908, 29906]}, {"type": "point", "id": "0x34c65fe2673b9559", "point": [81099, 40221, 23977]}, {"type": "point", "id": "0x224800863a7c437e", "point": [76408, 21362, 22562]}, {"type": "point", "id": "0x522b547201d30fc2", "point": [89208, 18548, 33472]}, {"type": "point", "id": "0x4bc2d7736c5b7e5", "point": [78398, 33256, 20886]}, {"type": "point", "id": "0x3fa669bf774e5d7b", "point": [91380, 

In [33]:
# Or add it programmatically to an existing neuroglancer state.
state = ngsk.parse_nglink('https://neuroglancer-demo.appspot.com/#!gs://flyem-male-cns/v0.9/male-cns-v0.9.json')
state['layers'].append(Tm9_soma_layer_json)
print(ngsk.encode_ngstate('https://neuroglancer-demo.appspot.com', state))

https://neuroglancer-demo.appspot.com/#!%7B%22title%22%3A%20%22MaleCNS%20v0.9%22%2C%20%22dimensions%22%3A%20%7B%22x%22%3A%20%5B8e-09%2C%20%22m%22%5D%2C%20%22y%22%3A%20%5B8e-09%2C%20%22m%22%5D%2C%20%22z%22%3A%20%5B8e-09%2C%20%22m%22%5D%7D%2C%20%22position%22%3A%20%5B48311.5%2C%2025295.765625%2C%2028511.5%5D%2C%20%22crossSectionScale%22%3A%2088.13122564876332%2C%20%22projectionOrientation%22%3A%20%5B0%2C%20-0.002181659685447812%2C%200%2C%200.999997615814209%5D%2C%20%22projectionScale%22%3A%2099656.51830908704%2C%20%22layers%22%3A%20%5B%7B%22type%22%3A%20%22image%22%2C%20%22source%22%3A%20%7B%22url%22%3A%20%22precomputed%3A//gs%3A//flyem-male-cns/em/em-clahe-jpeg%22%2C%20%22subsources%22%3A%20%7B%22default%22%3A%20true%7D%2C%20%22enableDefaultSubsources%22%3A%20false%7D%2C%20%22tab%22%3A%20%22rendering%22%2C%20%22name%22%3A%20%22em-clahe%22%7D%2C%20%7B%22type%22%3A%20%22segmentation%22%2C%20%22source%22%3A%20%5B%7B%22url%22%3A%20%22precomputed%3A//gs%3A//flyem-male-cns/v0.9/segmentation%2

In [36]:
long_link = ngsk.encode_ngstate('https://neuroglancer-demo.appspot.com', state)

import requests
SHORTENER_URL = 'https://shortng-bmcp5imp6q-uc.a.run.app'
r = requests.post(f'{SHORTENER_URL}/shortng', json={'text': Tm9_soma_layer_json})
r.content

b'<!doctype html>\n<html lang=en>\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'